# 5.Ajax 数据爬取

有时候我们在用 requests 抓取页面的时候，
得到的结果可能和在浏览器中看到的不一样：
在浏览器中可以看到正常显示的页面数据，但是使用requests得到的结果并没有。

这是因为requests获取的都是原始的HTML文档,
而浏览器中的页面则是经过JavaScript处理数据后生成的结果，
这些数据的来源有多种，可能是通过Ajax加载的，
可能是包含在HTML文档中的，
也可能是经过JavaScript和特定算法计算后生成的。

对于第一种情况，数据加载是一种异步加载方式，原始的页面最初不会包含某些数据，
原始页面加载完后，会再向服务器请求某个接口获取数据，
然后数据才被处理从而呈现到网页上，这其实就是发送了一个Ajax请求。

照Web发展的趋势来看，这种形式的页面越来越多。 网页的原始 HTML 文档不会包含任何数据，
数据都是通过Ajax统一加载后再呈现出来的，
这样在Web开发上可以做到前后端分离，而且降低服务器直接渲染页面带来的压力。

 所以如果遇到这样的页面，直接利用requests等库来抓取原始页面，
 是无法获取到有效数据的，
 这时需要分析网页后台向接口发送的Ajax请求，
 如果可以用**requests**库来模拟**Ajax**请求，那么就可以成功抓取了。

**Ajax，全称为 Asynchronous JavaScript and XML**，即异步的JavaScript和XML。 它不是一门编程语言，
而是利用 JavaScript 在保证页面不被刷新、
页面链接不改变的情况下与服务器交换数据并更新部分网页的技术。

有了**Ajax**，便可以在页面不被全部刷新的情况下更新其内容。
在这个过程中，
页面实际上是在后台与服务器进行了数据交互，
获取到数据之后，再利用**JavaScript**改变网页，这样网页内容就会更新了。

示例：[http://www.w3school.com.cn/ajax/ajax_xmlhttprequest_send.asp](http://www.w3school.com.cn/ajax/ajax_xmlhttprequest_send.asp)

[https://zyk.mingrisoft.com/Develop/view/id/1838/type/6/cid/45.html](https://zyk.mingrisoft.com/Develop/view/id/1838/type/6/cid/45.html)

Ajax加载在微博、QQ空间等比较常见

测试：打开新浪微博，进入到开发者选项，切换到Network选项卡，随后重新刷新页面，可以发现这里出现了非常多的条目

Ajax有其特殊的请求类型，它叫作xhr

如果显示的Request Headers中有一个信息为 X-Requested-With:XMLHttpRequest，这就标记了此请求是Ajax

图片显示![](AjaxExample.jpg)

的微博页面的真实数据并不是最原始的页面返回的，
而是后来执行JavaScript后再次向后台发送了Ajax请求，
浏览器拿到数据后再进一步渲染出来的。

分析出 Ajax请求的一些详细信息了，
接下来只需要用程序模拟这些**Ajax**请求，
就可以轻松提取我们所需要的信息了。

Ajax实际执行的代码：

Ajax 信息提取基本流程：
# 1. 分析请求

打开Ajax的XHR过滤器，然后一直滑动页面以加载新的微博内容。
可以看到，会不断有Ajax请求发出。

效果图：
![](Ajaxget.jpg)

# 2. 分析响应

观察这个请求的响应内容

# 3. 实战：分析Ajax爬取今日头条的风景美图并下载到本地！

爬取库：**requests**
流程：

## 1.爬取分析

今日头条首页：[http://www.toutiao.com/](http://www.toutiao.com/)

查看对应的请求与响应状态

我们的目的是要抓取其中的美图，
这里一组图就对应前面 data 字段中的一条数据。
每条数据还有一个**image_detail(image_list)**字段，它是列表形式，
这其中就包含了组图的所有图片列表

例子：![](toutiao.jpg)

因此我们只需提取Url并下载下来就好,一组图都建立文件夹，
文件夹的名称就为组图的标题。

接下来，就可以直接用Python来模拟这个Ajax请求，
然后提取出相关美图链接井下载。
但是在这之前，我们还需要分析一下**URL**的规律。

切换回Headers选项卡，观察一下它的请求URL和Headers信息

可以看到 ，这是一个GET请求，请求URL的参数有 offset、format等
![](query.jpg)

这里观察一下后续链接的参数，发现变化的参数只有 offset ，
其他参数都没有变化，而且第二次请求的offset值为20,
第三次为40，第四次为60，所以可以发现规律，
这个offset值就是偏移量，
进而可以推断出count参数就是一次性获取的数据条数。
因此，我们可以用offset参数来控制数据分页。
这样一来，我们就可以通过接口批量获取数据了

刷新一次，offset偏移值增加20

In [1]:
from urllib.parse import urlencode
import requests
def get_page(offset):
    params = {
        'offset':offset,
        'format':'json',
        'keyword':'风景',
        'autoload':'true',
        'count':'20',
        'en_qc':'1',
        'cur_tab':'1'
    }
    url = 'https://www.toutiao.com/api/search/content?'+urlencode(params)
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError:
        return None
for i in range(10):
    print(get_page(str(i*2)))
    print()

{'count': 0, 'return_count': 0, 'query_id': '6537509081622516995', 'has_more': 0, 'request_id': '202007252250020100140170211F25DFDD', 'search_id': '202007252250020100140170211F25DFDD', 'cur_ts': 1595688602, 'offset': 10, 'message': 'success', 'pd': 'synthesis', 'show_tabs': 1, 'keyword': '风景', 'city': '舟山', 'log_pb': {'impr_id': '202007252250020100140170211F25DFDD', 'is_incognito': 0}, 'data': None, 'data_head': [{'challenge_code': 0, 'cell_type': 71, 'keyword': '风景', 'url': 'sslocal://search?keyword=%E9%A3%8E%E6%99%AF&from=&source=search_tab'}], 'ab_fields': None, 'latency': 0, 'search_type': 2, 'tab_rank': None, 'temp_type': 0, 'tab_list': None, 'stability_error': 0, 'synthesis_stability_error': 0, 'gray_page_switch': False}

{'count': 0, 'return_count': 0, 'query_id': '6537509081622516995', 'has_more': 0, 'request_id': '202007252250010100140500121650368E', 'search_id': '202007252250010100140500121650368E', 'cur_ts': 1595688602, 'offset': 12, 'message': 'success', 'pd': 'synthesis', 

这里我们用**urlencode()**方法构造请求的GET参数，
然后用requests请求这个链接，如果返回状态码为200，
则调用response的json()方法将结果转为**JSON**格式，然后返回。

## 接下来，再实现一个解析方法：

提取每条数据的image_detail(image_list) 字段中的每一张图片链接，
将图片链接和图片所属的标题一并返回，此时可以构造一个生成器。

接下来，实现一个保存图片的方法 save_image()，
其中 item 就是前面 get_images()方法返回的一个字典。
在该方法中，首先根据 item 的 title 来创建文件夹，
然后请求这个图片链接，获取图片的二进制数据，
以二进制的形式写入文件。

In [ ]:
from urllib.parse import urlencode
import requests
import os
originalocation = os.getcwd()#先确定当前路径
# print("local rount: "+originalocation)
count = 0
def get_page(offset):
    params = {
        'offset':offset,
        'format':'json',
        'keyword':'风景',
        'autoload':'true',
        'count':'20',
        'cur_tab':'1',
    }
    url = 'https://www.toutiao.com/api/search/content?'+urlencode(params)
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError:
        return None
# for i in range(10):
#     print(get_page(str(i*2)))
#     print()


def get_image(json):
    print(json.get('data'))
    if json.get('data'):
        for item in json.get("data"):
            title = item.get('title')
            print(title)
            images = item.get('image_list')
            if not images == None:
                for image in images:
                    yield {
                        'image': image.get('url'),
                        'title': title
                    }

def save_image(item):
    """存储相关的图片"""
    print("存储图片：")
    global count
    if not os.path.exists(item.get('title')):
        os.mkdir(item.get('title'))
        # os.chdir(os.getcwd()+item.get('title'))
    try:
        response = requests.get(item.get('image'))
        if response.status_code == 200:
            filepath = item.get('title')+"/"+str(count)+".jpg"
            if not os.path.exists(filepath):
                with open(filepath,'wb') as f:
                    f.write(response.content)
                count+=1
            else:
                print('Already download')
        # os.chdir(originalocation)
    except requests.ConnectionError as e:
        print("Error.\n"+str(e))
from multiprocessing.pool import Pool
import time
def main(offset):
    #print("mainfunction......")
    json = get_page(offset)
    #print(json)
    for i in get_image(json):
        print(i)
        save_image(i)

START = 1
END =20
if __name__ =='__main__':
    print('start......')
    pol = Pool()
    time.sleep(1)
    groups =[x*20 for x in range(START,END+1)]
    pol.map(main,groups)
    pol.close()
    pol.join()
    print("over......")


注意：爬取的时候，因为访问过于频繁可能导致无法爬出数据，因此最好添加代理服务！

